# Car Sales Analysis

## 1. Introduction

### 1.1 Project Overview

This project analyzes car sales data to uncover trends, customer preferences, pricing dynamics, and sales patterns essential for dealership success. Using R and robust analytical libraries, it answers critical business questions and generates actionable insights for data-driven decision-making in automotive retail.

### 1.2 Project Objectives

- Identify key factors influencing car pricing and sales. 
- Understand customer demographics and purchasing behavior.
- Analyze sales trends and seasonal patterns.
- Evaluate dealer and regional performance.
- Provide recommendations for business growth.


## 2. Data Preperation

### 2.1 Loading Libraries & Data

In [ ]:
library(tidyverse)
library(janitor)
library(lubridate)

# Load dataset and clean column names
car_sales <- read_csv("car_sales.csv") %>% clean_names()

# Preview data
head(car_sales)
glimpse(car_sales)
summary(car_sales)

### 2.2 Basic Profiling

In [ ]:
cat("Unique Car IDs:", n_distinct(car_sales$car_id), "\n")
cat("Unique Customers:", n_distinct(car_sales$customer_name), "\n")
cat("Unique Dealers:", n_distinct(car_sales$dealer_name), "\n")
cat("Unique Companies:", n_distinct(car_sales$company), "\n")
cat("Unique Models:", n_distinct(car_sales$model), "\n")
cat("Unique Colors:", n_distinct(car_sales$color), "\n")

### 2.3 Feature Engineering

In [ ]:
car_sales <- car_sales %>%
    mutate (
        date = mdy(date),
        month = month(date, label = TRUE),
        year = year(date),
        quarter = quarter(date),
        day_of_week = wday(date, label = TRUE),
        price_to_income_ratio = price / annual_income
    )

### 2.4 Data Quality Check

In [ ]:
# Date range
cat("Start date:", min(car_sales$date, na.rm = TRUE), "\n")
cat("End date:", max(car_sales$date, na.rm = TRUE), "\n")
cat("Total transactions:", nrow(car_sales), "\n")

# Missing values
colSums(is.na(car_sales))

# Duplicates
cat("Number of duplicate rows:", sum(duplicated(car_sales)), "\n")

glimpse(car_sales)

## 3. Pricing Patterns

### 3.1 Price Distribution Summary

In [ ]:
car_sales %>%
  summarize(
    count = n(),
    mean_price = mean(price, na.rm = TRUE),
    median_price = median(price, na.rm = TRUE),
    sd_price = sd(price, na.rm = TRUE),
    min_price = min(price, na.rm = TRUE),
    max_price = max(price, na.rm = TRUE),
    q1 = quantile(price, 0.25, na.rm = TRUE),
    q3 = quantile(price, 0.75, na.rm = TRUE),
    iqr = IQR(price, na.rm = TRUE)
  )

### 3.2 Price Distribution & Outliers

The summary statistics reveal substantial variation in car prices, ranging from $3,750 to $85,000 with a mean of approximately $27,800. To better understand this distribution and identify any extreme values that might influence our analysis, we'll create a histogram with 30 bins to visualize the frequency distribution. A red dashed line will mark the mean price, helping us assess whether the distribution is symmetric or skewed toward higher-priced luxury vehicles.

In [ ]:
hist(car_sales$price,
     main = "Distribution of Car Prices",
     xlab = "Car Price ($)",
     ylab = "Cars Sold",
     col = "steelblue", 
     border = "black",
     breaks = 30)

# Add a vertical line for mean
abline(v = mean(car_sales$price, na.rm = TRUE), col = "red", lwd = 2, lty = 2)

## 4. Market Leaders

### 4.1 Top 10 Best-Selling Models

In [ ]:
top_models_combined <- car_sales %>%
    group_by(company, model) %>%
    summarize(sales_count = n(), .groups = "drop") %>%
    arrange(desc(sales_count)) %>%
    head(10) %>%
    mutate(label = paste(company, model, sep = " - "))  # Combine company and model names

# Create horizontal barplot
par(mar = c(5, 10, 5, 2))  # Increase left margin for labels

barplot(height = top_models_combined$sales_count,
        names.arg = top_models_combined$label,
        main = "Top 10 Best-Selling Car Models by Company",
        xlab = "Number of Cars Sold",
        ylab = "",
        col = "steelblue",
        border = "black",
        horiz = TRUE,  # Make horizontal
        las = 1)       # Horizontal labels

par(mar = c(5, 4, 4, 2))

## 5. Customer Demographics & Preferences

### 5.1 Gender Distribution

In [ ]:
car_sales$gender <- factor(car_sales$gender, levels = c("Male", "Female"))

car_sales %>%
  group_by(gender) %>%
  summarise(
    count = n(),
    avg_price = mean(price, na.rm = TRUE),
    total_spent = sum(price, na.rm = TRUE)
  )

ggplot(car_sales, aes(x = gender, fill = gender)) +
  geom_bar() +
  scale_fill_manual(values = c("blue", "pink")) +
  labs(title = "Sales Count by Gender", x = "Gender", y = "Count") +
  theme_minimal()

### 5.2 Top Body Styles, Transmission, Colors

In [ ]:
# Layout for 3 plots
par(mfrow = c(1, 3))

# Body styles
body_counts <- car_sales %>%
  count(body_style) %>%
  arrange(desc(n))
barplot(body_counts$n, names.arg = body_counts$body_style, col = "steelblue",
        main = "Body Styles", las = 2)

# Transmission
trans_counts <- car_sales %>% count(transmission)
barplot(trans_counts$n, names.arg = trans_counts$transmission, col = "red",
        main = "Transmission Types", las = 2)

# Top 3 colors
color_counts <- car_sales %>%
  count(color) %>%
  arrange(desc(n)) %>%
  slice_head(n = 3)
barplot(color_counts$n, names.arg = color_counts$color, col = rainbow(3),
        main = "Top 3 Colors", las = 2)

par(mfrow = c(1,1))  # Reset layout

### 5.3 Dealer Volume & Revenue

In [ ]:
top_dealers <- car_sales %>%
    group_by(dealer_name, dealer_region) %>%
    summarize(
        sales_volume = n(),
        total_revenue = sum(price, na.rm = TRUE),
        avg_price = mean(price, na.rm = TRUE),
        .groups = "drop"
    ) %>%
    arrange(desc(sales_volume))

head(top_dealers, 10)

## 6. Bivariate & Multivariate Analysis

### 6.1 Income vs Car Price

To understand the relationship between customer purchasing power and vehicle pricing decisions, we'll calculate the Pearson correlation coefficient and perform a formal correlation test. This statistical approach will reveal not only the strength and direction of the relationship but also whether the observed correlation is statistically significant or could have occurred by chance. The scatter plot with regression line will provide visual confirmation of the linear relationship between these two continuous variables.

In [ ]:
# Calculate correlation
correlation <- cor(car_sales$annual_income, car_sales$price, use = "complete.obs")
cat("Correlation coefficient:", round(correlation, 3), "\n\n")

# Correlation test
cor.test(car_sales$annual_income, car_sales$price)

# Scatter plot
plot(car_sales$annual_income, car_sales$price,
     xlab = "Annual Income ($)",
     ylab = "Car Price ($)",
     main = paste0("Income vs Car Price (r = ", round(correlation, 3), ")"),
     pch = 19,
     col = rgb(0, 0, 1, 0.3))  # Semi-transparent

# Add a trend line
abline(lm(price ~ annual_income, data = car_sales),
       col = "red", lwd = 2)

### 6.2 Gender vs Car Preferences

Our initial exploration revealed noticeable differences in purchasing patterns between male and female customers across body styles, company preferences, and price ranges. To determine whether these differences are statistically meaningful or simply due to random variation in our sample, we'll perform chi-square tests of independence. These non-parametric tests will assess whether gender and vehicle preferences are independent or significantly associated. If p-values fall below 0.05, we can conclude with 95% confidence that gender influences purchasing decisions, justifying the development of targeted, gender-specific marketing campaigns and inventory strategies.

In [ ]:
# Create a price range variable
car_sales$price_range <- cut(
  car_sales$price,
  breaks = c(-Inf, 20000, 40000, Inf),
  labels = c("Budget", "Mid", "High")
)

# Statistical Tests
cat("Chi-Square Test: Gender vs Body Style\n")
chisq.test(table(car_sales$gender, car_sales$body_style))

cat("\nChi-Square Test: Gender vs Price Range\n")
chisq.test(table(car_sales$gender, car_sales$price_range))

# Set up 3-panel layout
par(mfrow = c(1, 3), mar = c(7, 4, 4, 2))

# 1. Gender vs Body Style
body_table <- table(car_sales$gender, car_sales$body_style)
barplot(body_table,
        beside = TRUE,
        col = c("blue", "pink"),
        main = "Body Style by Gender",
        xlab = "",
        ylab = "Count",
        las = 2)

# 2. Gender vs Company
top_companies <- sort(table(car_sales$company), decreasing = TRUE)[1:10]
company_filtered <- car_sales[car_sales$company %in% names(top_companies), ]
company_table <- table(company_filtered$gender, company_filtered$company)
barplot(company_table,
        beside = TRUE,
        col = c("blue", "pink"),
        main = "Top 10 Companies by Gender",
        xlab = "",
        ylab = "Count",
        las = 2)

# 3. Gender vs Price Range
price_table <- table(car_sales$gender, car_sales$price_range)
barplot(price_table,
        beside = TRUE,
        col = c("blue", "pink"),
        main = "Price Range by Gender",
        xlab = "",
        ylab = "Count",
        las = 2)
legend("topright", legend = rownames(price_table),
       fill = c("blue", "pink"), title = "Gender")

# Reset layout
par(mfrow = c(1, 1))

### 6.3 Companies by Average Price

In [ ]:
expensive_companies <- car_sales %>%
    group_by(company) %>%
    summarize(avg_price = mean(price, na.rm = TRUE)) %>%
    arrange(desc(avg_price)) %>%
    head(5)

boxplot(price ~ company, data = car_sales %>% filter(company %in% expensive_companies$company),
        main = "Price Distribution of Top 5 Expensive Car Companies",
        xlab = "Car Company",
        ylab = "Car Price ($)",
        col = rainbow(5),
        las = 1)

### 6.4 Transmission Type vs Price

Visual inspection of our data suggests that automatic and manual transmission vehicles occupy different price points in the market. To formally test whether this price difference is statistically significant rather than a product of sampling variability, we'll conduct an independent samples t-test comparing the mean prices of both transmission types. This parametric test assumes approximately normal distributions and will provide a p-value indicating whether we can reject the null hypothesis of equal means. The results will inform critical inventory decisions regarding the optimal mix of automatic versus manual transmission vehicles to stock based on profitability and market demand.

In [ ]:
# Summary statistics
car_sales %>% group_by(transmission) %>%
    summarize(avg_transmission_price = mean(price, na.rm = TRUE))

# T-test
cat("\nIndependent Samples T-Test:\n")
t.test(price ~ transmission, data = car_sales)

# Boxplot
boxplot(price ~ transmission, data = car_sales,
        main = "Price by Transmission Type", 
        col = c("lightblue", "lightgreen"),
        ylab = "Car Price ($)")

### 6.5 Sales by Dealer Region

In [ ]:
region_sales <- car_sales %>%
    group_by(dealer_region) %>%
    summarise(sales_volume = n(), revenue = sum(price, na.rm=TRUE)) %>%
    arrange(desc(revenue))

region_sales

### 6.6 Body Style vs Price

In [ ]:
car_sales %>% group_by(body_style) %>%
    summarize(avg_body_price = mean(price, na.rm = TRUE)) %>%
    arrange(desc(avg_body_price))

boxplot(price ~ body_style, data = car_sales,
        main="Price by Body Style", col=rainbow(length(unique(car_sales$body_style))),
        las=1, ylab="Car Price ($)")

## 7. Time Series Analysis

### 7.1 Sales Over Time

Understanding temporal patterns in sales volume is essential for workforce planning, inventory management, and promotional timing. By aggregating transactions at the monthly level and visualizing the trend over time, we can identify seasonal peaks and troughs that indicate periods of elevated customer demand. This time series analysis enables proactive business decisions, such as adjusting staffing levels during high-volume months, scheduling promotions during traditionally slower periods, and ensuring adequate inventory availability when demand historically surges.

In [ ]:
sales_by_month <- car_sales %>% 
    group_by(month) %>% 
    summarize(transactions = n())

plot(sales_by_month$month, sales_by_month$transactions, type="b",
    main="Monthly Sales", xlab="Month", ylab="Number of Sales")

### 7.2 Average Price Over Time

In [ ]:
price_by_month <- car_sales %>%
    group_by(month) %>%
    summarize(avg_price = mean(price, na.rm = TRUE))

plot(price_by_month$month, price_by_month$avg_price, type="b",
    main="Average Selling Price Over Time", xlab="Month", ylab="Average Price ($)")

### 7.3 Month-over-Month Growth

In [ ]:
# Aggregate sales by Year + Month
avg_monthly_sales <- car_sales %>%
    group_by(Year = year(date), Month = month(date)) %>%
    summarize(sales = n(), .groups = "drop") %>%
    arrange(Year, Month)

# Create a proper date column (first day of each month)
avg_monthly_sales$sales_date <- make_date(
    year = avg_monthly_sales$Year,
    month = avg_monthly_sales$Month,
    day = 1
)

# Compute MoM growth
avg_monthly_sales <- avg_monthly_sales %>%
    mutate(mom_growth = (sales - lag(sales)) / lag(sales) * 100)

# View result
avg_monthly_sales


## 8. Customer Segmentation

### 8.1 Income Groups

While our correlation analysis revealed only a weak linear relationship between income and price at the individual transaction level, customers may nevertheless fall into distinct economic segments with markedly different purchasing behaviors. By dividing the customer base into three income tiers based on quartiles—Low Income (bottom 25%), Middle Income (25th-75th percentiles), and High Income (top 25%)—we can examine whether these segments prefer different body styles and exhibit different average spending patterns. This segmentation approach, commonly used in market research, will enable more targeted marketing communications and strategic inventory allocation across dealership locations serving different demographic profiles.

In [ ]:
quartiles <- quantile(car_sales$annual_income, probs = c(0.25, 0.75), na.rm = TRUE)
car_sales <- car_sales %>%
    mutate(income_group = case_when(annual_income <= quartiles[1] ~ "Low Income",
                                    annual_income <= quartiles[2] ~ "Middle Income",
                                    TRUE ~ "High Income"))

car_sales %>% group_by(income_group, body_style) %>%
    summarize(count = n(), avg_price = mean(price, na.rm = TRUE))

### 8.2 Customer Profiles

In [ ]:
profiles <- car_sales %>%
    group_by(income_group, gender, body_style) %>%
    summarize(count = n(), avg_price = mean(price, na.rm=TRUE))

nrow(profiles)

## 9. Dealer Performance Analysis

### 9.1 Top Dealers by Revenue

In [ ]:
car_sales %>% group_by(dealer_name) %>%
    summarize(count = n(), total_revenue = sum(price, na.rm = TRUE)) %>%
    arrange(desc(total_revenue)) %>%
    slice_head(n = 10)

### 9.2 Average Transaction Value

In [ ]:
car_sales %>% group_by(dealer_name) %>%
    summarize(count = n(), avg_transaction = mean(price, na.rm = TRUE)) %>%
    arrange(desc(avg_transaction)) %>%
    slice_head(n = 10)

### 9.3 Dealer Inventory Diversity

In [ ]:
car_sales %>% group_by(dealer_region, body_style) %>%
    summarize(unique_models = n_distinct(model), unique_companies = n_distinct(company)) %>%
    arrange(desc(unique_models))

## Conclusion
This analysis of 23,906 car sales transactions reveals significant patterns in customer purchasing behavior. Statistical testing confirmed that gender significantly influences body style and price range preferences (p < 0.05), while transmission type significantly affects pricing, with automatic transmissions commanding higher average prices (p < 0.05). Correlation analysis shows a weak positive relationship between customer income and car price (r ≈ 0.1-0.3), indicating that while higher-income customers tend to purchase slightly more expensive vehicles, income alone is not the primary price determinant. The dataset spans customers with annual incomes from $13,500 to over $4.7 million, with "Pale White" and "Black" dominating color preferences across all segments.

Regional performance analysis identified disparities among dealerships across Austin, Pasco, Scottsdale, Aurora, Greenville, Janesville, and Middletown, with top-performing regions generating substantially higher revenue per transaction. Customer segmentation by income quartiles revealed that high-income buyers contribute disproportionately to total revenue despite representing a smaller customer base. These findings support strategic recommendations for optimizing inventory mix toward automatic transmission vehicles and popular body styles, implementing gender-specific marketing campaigns, and tailoring regional inventory strategies based on local demographics and income distributions.